# Imports

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS

This notebook takes an ALFALFA grid (as hosted in the NRAO archive) and updates the header keywords. This ensures the headers are accurate and compatible with modern standards, e.g., enabling spectral axis conversions in CARTA. 

Also note that the original WCS in the header can produce a slightly inaccurate projection and we therefore provide an approximate WCS solution that should be used for most applications. The extracted positions of published ALFALFA sources were corrected as described in [Brian Kent's thesis](https://ecommons.cornell.edu/items/a278d737-b863-4e07-b637-88587dff669f), but these correction were never propagated back to the grids themselves.

All of these header updates are incoporated into the [load_grid](../scripts/load_grid.py) function that we recommend using to load ALFALFA data cubes into Python. However, this notebook may be useful if you wish to update the FITS files themselves for use with other software.

**Note**: You will need to have already downloaded the grid you wish to use and placed it in the same directory as this notebook. You can find instructions for accessing the grids in the [grid_access.md](../docs/grid_access.md) file in the docs folder and illustrated instructions on the [wiki](https://github.com/jonesmg/ALFALFA_Legacy/wiki/Grid-access-via-NRAO-archive). In this case you need the 1044+13 grid and the "a" spectral cube needs to be placed in the current working directory.

# Read in data

In [ ]:
# Define the grid you are using
grid_ra = '1044'
grid_dec = '13'
freq_slice = 'a'

grid_filename = f'{grid_ra}+{grid_dec}{freq_slice}_spectral.fits'

# open the grid
cube_use =  fits.open(grid_filename)

In [ ]:
# open the header
header_new = cube_use["PRIMARY"].header.copy()

# Modifying Headers

In [ ]:
# view the current header
header_new

Change the keywords for the frequency axis, and add rest-frame frequency.

In [ ]:
header_new["CTYPE3"]  = "FREQ"
header_new["CUNIT3"]  = "MHz"
header_new["SPECSYS"] = "HELIOCEN"

header_new["RESTFRQ"] = 1420.405751768e6  # (Hz)

header_new["CNAME3"]  = "FREQ-HEL"

Add keywords for beam

In [ ]:
header_new["BMAJ"]  = 3.8/60
header_new["BMIN"]  = 3.3/60
header_new["BPA"]   = 0

Change the stokes parameters to LL and RR

In [ ]:
header_new["CRVAL4"] = -2  # LL and RR

Change BUNIT so it can be parsed by astropy.unit

In [ ]:
header_new["BUNIT"] = "mJy/beam"

# Update the WCS

In [ ]:
cen_ra = (float(grid_ra[:2]) + float(grid_ra[2:])/60)*15
cen_dec = float(grid_dec)

header_new['CRVAL1'] = cen_ra
header_new['CDELT1'] = -1/60 #degrees
header_new['CRPIX1'] = 72.5
header_new['CRVAL2'] = cen_dec
header_new['CDELT2'] = 1/60 #degrees
header_new['CRPIX2'] = 72.5

# Write fits

In [ ]:
# attach the new header to the data
cube_use["PRIMARY"].header = header_new

In [ ]:
# define the output grid name
out_ext = 'new'
grid_out_filename = f'{grid_ra}+{grid_dec}{freq_slice}_spectral_{out_ext}.fits'

# write the new grid out
cube_use.writeto(grid_out_filename)